In [25]:
# !pip install bs4

In [2]:
import urllib.request
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import numpy as np
import io
import json
import pandas as pd
import time
import urllib.parse

In [27]:
# crawling from a search bar
# returns list of urls of product in search result
def search_bar_crawler(link, crawl_limit):
    html = urlopen(link)
    soup = BeautifulSoup(html.read(), 'html.parser')

    # with io.open('html.txt', 'w', encoding='utf8') as file:
    #     file.write(str(soup))

    # need to compile a list of item href
    script_tags = soup.find_all("script", type="application/ld+json")

    # Create a list to store the product URLs
    product_urls = []

    # Iterate over the `<script type="application/ld+json">` tags
    for script_tag in script_tags:

        # Extract the JSON text from the `<script type="application/ld+json">` tag
        json_text = script_tag.text

        # Load the JSON text into a Python object
        json_object = json.loads(json_text)

        # Check if the JSON object has the "@type": "Product" property
        if json_object["@type"] == "Product":

            # Get the "href" attribute of the JSON object
            product_url = json_object["url"]

            # Add the product URL to the list
            product_urls.append(product_url)
        
        # arbitrary stop sign (i can think of a million ways this can go wrong)
        # 10 is too much
        if len(product_urls) == crawl_limit:
            return product_urls
    
    return product_urls

# print(search_bar_crawler('https://tiki.vn/search?q=adidas'))



In [28]:
# specific product crawling (categories)
# returns all category tags from list of urls
def get_tiki_category(product_urls):
    category_tags = []
    count = 0
    for product_url in product_urls:
        # arbitrary stop sign to avoid RemoteDisconnected

        productHTML = urlopen(product_url)
        soup = BeautifulSoup(productHTML.read(), 'html.parser')

        breadcrumb_items = soup.find_all("a", class_="breadcrumb-item")

        # Extract all spans within the <a> tags
        breadcrumb_spans = []
        for breadcrumb_item in breadcrumb_items:
            breadcrumb_spans.extend(breadcrumb_item.find_all("span"))

        breadcrumb_texts = [bc.text for bc in breadcrumb_items[0: len(breadcrumb_items) - 1]]

        # Print the breadcrumb spans
        category_tag = ' > '.join(breadcrumb_texts)

        # print(category_tag)
        category_tags.append(category_tag)
    
    return category_tags

# adidas_category_tags = get_tiki_category(search_bar_crawler('https://tiki.vn/search?q=adidas'))
# print(adidas_category_tags)







In [29]:
# a really simple function to count the occurences of each category tag
def list_to_dict_count(list1):
    """Returns a dictionary with unique elements of the given list as keys and their count of occurrence as values.

    Args:
        list1: A Python list.

    Returns:
        A Python dictionary.
    """

    dict1 = {}
    for element in list1:
        if element in dict1:
            dict1[element] += 1
        else:
            dict1[element] = 1
    return dict1

# print(list_to_dict_with_unique_elements_and_count(adidas_category_tags))



In [38]:
# compile the datasheet into corresponding classes (build by dict)
def compile_df_to_dict_with_unique_label_values(df):
    # Get the unique label values.
    unique_label_values = df['Label'].unique()

    # Create a dictionary to store the results.
    results = {}

    # Iterate over the unique label values and add them to the dictionary.
    for label_value in unique_label_values:
        # Get the product name for the current label value.
        product_name = df.loc[df['Label'] == label_value, 'Product Name'].tolist()

        # Add the label value and product name to the dictionary.
        results[label_value] = product_name

    # Return the dictionary.
    return results


# Example usage:

df = pd.read_excel('data_output.xlsx')
# unique_labels = set(df['Label'])
# print(len(unique_labels))

# Compile the DataFrame into a dictionary.
labelDict = compile_df_to_dict_with_unique_label_values(df)


# Print the dictionary.
# print(len(labelDict))

['HB Gift Box 1 - Bia đóng hộp làm quà tặng sang trọng']


In [39]:
# link_format = 'https://tiki.vn/search?q='
# categs = []

# for product in labelDict['__Ghế_văn_phòng'][0:4]:
#     encoded_string = urllib.parse.quote_plus(product)
#     link = link_format + encoded_string
#     categ = get_tiki_category(search_bar_crawler(link))
#     categs.extend(categ)

# # there should be 10 per link
# print(len(categs))

['Chụp khí Hypertherm 220977']


In [32]:
# dict_count = list_to_dict_count(categs)
# print(len(categs))


# sorted_dict = sorted(dict_count.items(), key=lambda item: item[1], reverse= True)

# # Print the sorted dictionary
# print(sorted_dict)
# most_popular_categ = sorted_dict[0][0]
# pop_ratio = sorted_dict[0][1] / len(categs)
# second_most_popular_categ = sorted_dict[1][0]
# second_pop_ratio = sorted_dict[1][1] / len(categs)
# print(most_popular_categ)
# print(pop_ratio)
# print(second_most_popular_categ)
# print(second_pop_ratio)


first wave of tiki scraping

strengths:
    all labels (1971) have been looked into
    
weaknesses:
    those of low ratio

    duplicates

    -> needs to be compiled and analyzed via pandas
    
    theres too many of 'em, redo it

In [33]:

# to ensure that I don't have to read the same thing all over, I check all the read labels, and remove them from the labelDict

# find list of read labels
# with open('categories_ratio.txt', 'r', encoding='utf-8') as f:
#     lines = f.readlines()

# labelLines = []
# for i, line in enumerate(lines):
#     if i % 3 == 0:
#         labelLines.append(line.replace('\n', ''))

# # print(label_lines)

# unreadLabelDict = {}
# for key, value in labelDict.items():
#     if key not in labelLines:
#         unreadLabelDict[key] = value 

# print(unreadLabelDict)
# link_format = 'https://tiki.vn/search?q='
# for key in unreadLabelDict:
#     categs = []
#     count = 0
#     for product in unreadLabelDict[key]:
#         # dumbest error ever
#         encoded_string = urllib.parse.quote_plus(product.replace('∅', ''))
#         link = link_format + encoded_string
#         # print(link)
#         categ = ""
#         try:
#             categ = get_tiki_category(search_bar_crawler(link))
#         except:
#             time.sleep(5)
#             categ = get_tiki_category(search_bar_crawler(link))
            
#         categs.extend(categ)
#         if count >= 20:
#             time.sleep(3)
#             count = 0
#         else:
#             count += len(categ)
    
#     dict_count = list_to_dict_count(categs)
#     sorted_dict = sorted(dict_count.items(), key=lambda item: item[1], reverse= True)

#     # Print the sorted dictionary
#     most_popular_categ = sorted_dict[0][0]
#     pop_ratio = sorted_dict[0][1] / len(categs)
#     with open('categories_ratio.txt', 'a', encoding='utf-8') as f:
#         f.write(str(key) + '\n')
#         f.write(most_popular_categ + '\n')
#         f.write(str(pop_ratio) + '\n')
        

    

"\nfirst wave of tiki scraping\nstrengths:\n    all labels (1971) have been looked into\nweaknesses:\n    those of low ratio\n    duplicates\n    -> needs to be compiled and analyzed via pandas\n    theres too many of 'em, redo it\n"

A quick program pushing categories_ratio.txt into a xlsx, for ease of analysis

failed, bad data

In [34]:

# with open('categories_ratio.txt', 'r', encoding= 'utf-8') as f:
#     lines = f.readlines()
#     labels = []
#     categs = []
#     weight = []
#     for i, line in enumerate(lines):
#         if i % 3 == 0:
#             labels.append(line.replace('\n', ''))
#         elif i % 3 == 1:
#             categs.append(line.replace('\n', ''))
#         else:
#             weight.append(line.replace('\n', ''))
    
#     weights = [float(x) for x in weight]
#     print(len(weights))

#     df = pd.DataFrame(list(zip(labels, categs, weights)), columns= ['Label', 'Category', 'Weight'])
#     df.to_excel('first_wave_tiki_crawl.xlsx')
    

'\nA quick program pushing categories_ratio.txt into a xlsx, for ease of analysis\nfailed, bad data\n'

In [35]:
with open('categories_ratio_wave_2.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# add another column: size of label

labelLines = []
for i, line in enumerate(lines):
    if i % 4 == 0:
        labelLines.append(line.replace('\n', ''))

# print(label_lines)

unreadLabelDict = {}
for key, value in labelDict.items():
    if key not in labelLines:
        unreadLabelDict[key] = value 



0


wave 2, re-crawling

In [36]:
# link_format = 'https://tiki.vn/search?q='
# for i, key in enumerate(unreadLabelDict):
    
#     categs = []
#     count = 0
#     limit = 20
#     HTTPErrorCount = 0
#     IndexErrorCount = 0
#     UnicodeErrorCount = 0
#     if (len(unreadLabelDict[key]) >= 50):
#         limit = 5
#     elif (len(unreadLabelDict[key]) >= 20):
#         limit = 10

#     for product in unreadLabelDict[key]:
#         # dumbest error ever
#         encoded_string = urllib.parse.quote_plus(product.replace('∅', ''))
#         link = link_format + encoded_string
#         link = link.replace('∅', '')
#         # print(link)
#         categ = ""

#         try:
#             categ = get_tiki_category(search_bar_crawler(link, crawl_limit= limit))
#         except HTTPError:
#             print(key + ' HTTP ERROR: ' + link)
#             HTTPErrorCount += 1
#             print(HTTPErrorCount)
#             continue
#         except UnicodeEncodeError:
#             print(key + ' UNICODE ERROR: ' + link)
#             UnicodeErrorCount += 1
#             print(UnicodeErrorCount)
#             continue
#         except:
#             time.sleep(5)
#             categ = get_tiki_category(search_bar_crawler(link, crawl_limit= limit))
            
#         categs.extend(categ)
#         if count >= 20:
#             time.sleep(3)
#             count = 0
#         else:
#             count += len(categ)
        
    
#     dict_count = list_to_dict_count(categs)
#     sorted_dict = sorted(dict_count.items(), key=lambda item: item[1], reverse= True)

#     # Print the sorted dictionary
#     try:
#         most_popular_categ = sorted_dict[0][0]
#         pop_ratio = sorted_dict[0][1] / len(categs)
#         with open('categories_ratio_wave_2.txt', 'a', encoding='utf-8') as f:
#             f.write(str(key) + '\n')
#             f.write(str(len(unreadLabelDict[key])) + '\n')
#             f.write(most_popular_categ + '\n')
#             f.write(str(pop_ratio) + '\n')
#     except IndexError:
#             print(key + ' INDEX ERROR: ' + link)
#             IndexErrorCount += 1
#             print(IndexErrorCount)
#             continue

Compiling second wave results into another excel file

In [4]:
# with open('categories_ratio_wave_2.txt', 'r', encoding= 'utf-8') as f:
#     lines = f.readlines()
#     labels = []
#     label_size = []
#     categs = []
#     weight = []
#     for i, line in enumerate(lines):
#         if i % 4 == 0:
#             labels.append(line.replace('\n', ''))
#         elif i % 4 == 1:
#             label_size.append(line.replace('\n', ''))
#         elif i % 4 == 2:
#             categs.append(line.replace('\n', ''))
#         else:
#             weight.append(line.replace('\n', ''))
    
#     weights = [float(x) for x in weight]
#     label_sizes = [int(x) for x in label_size]
#     print(len(weights))

#     df = pd.DataFrame(list(zip(labels, label_sizes, categs, weights)), columns= ['Label', 'Label Size', 'Category', 'Weight'])
#     df.to_excel('second_wave_tiki_crawl.xlsx')

1971
1971


In [8]:
df = pd.read_excel('second_wave_tiki_crawl.xlsx')
print(df.head(5))
print(len(df['Category'].unique()))


   Unnamed: 0                        Label  Label Size  \
0           0              __Ghế_văn_phòng         129   
1           1                  __Thước_dây           8   
2           2                    __Cưa_tay           2   
3           3                   __Váy,_đầm         158   
4           4  __Dụng_cụ_đo,_kiểm_tra_khác          55   

                                            Category    Weight  
0  Trang chủ > Nhà Cửa - Đời Sống > Nội thất > Nộ...  0.759690  
1  Trang chủ > Nhà Cửa - Đời Sống > Sửa chữa nhà ...  1.000000  
2  Trang chủ > Nhà Cửa - Đời Sống > Sửa chữa nhà ...  0.250000  
3  Trang chủ > Thời trang nữ > Đầm nữ > Đầm Dáng Xòe  0.437975  
4  Trang chủ > Nhà Cửa - Đời Sống > Sửa chữa nhà ...  0.370909  
776
